In [1]:
import torch
from diffusers import AutoencoderKLLTXVideo

single_file_url = "https://huggingface.co/Lightricks/LTX-Video/ltx-video-2b-v0.9.safetensors"
# transformer = LTXVideoTransformer3DModel.from_single_file(
#   single_file_url, torch_dtype=torch.bfloat16
# )
vae = AutoencoderKLLTXVideo.from_single_file(single_file_url, torch_dtype=torch.bfloat16)
# pipe = LTXImageToVideoPipeline.from_pretrained(
#   "Lightricks/LTX-Video", transformer=transformer, vae=vae, torch_dtype=torch.bfloat16
# )

ltx-video-2b-v0.9.safetensors:   0%|          | 0.00/9.37G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

In [8]:
import os
import torch
import imageio
import numpy as np
from torchvision import transforms
from typing import Tuple, List, Dict, Type
import logging

logging.getLogger('imageio_ffmpeg').setLevel(logging.ERROR)

# Configuration Class
class Config:
    def __init__(
        self,
        model_class: Type,
        model_path: str,
        device: str = "cuda",
        dtype: str = "float16",
        batch_size: int = 1,
        custom_batch_size: Dict[str, int] = None,
        source_base: str = "../resources/videos/",
        output_base: str = "./output_videos/",
        model_type: str = "general"  # "general" or "cosmos"
    ):
        self.model_class = model_class
        self.model_path = model_path
        self.device = device
        self.dtype = torch.float16 if dtype == "float16" else torch.bfloat16
        self.batch_size = batch_size
        self.custom_batch_size = custom_batch_size or {}
        self.source_base = source_base
        self.output_base = output_base
        self.model_type = model_type

# GeneralAutoEncoderKL Class
class GeneralAutoEncoderKL:
    def __init__(self, config: Config):
        self.config = config
        self.device = torch.device(self.config.device)
        self.dtype = self.config.dtype

        os.makedirs(self.config.output_base, exist_ok=True)

        if self.config.model_type == "general":
            self.model = self.config.model_class.from_pretrained(
                self.config.model_path,
                torch_dtype=self.dtype
            ).to(self.device)
            self.model.enable_slicing()
            self.model.enable_tiling()
        elif self.config.model_type == "cosmos":
            from cosmos_tokenizer.video_lib import CausalVideoTokenizer
            self.encoder = CausalVideoTokenizer(
                checkpoint_enc=f'{self.config.model_path}/encoder.jit'
            ).to(self.device)
            self.decoder = CausalVideoTokenizer(
                checkpoint_dec=f'{self.config.model_path}/decoder.jit'
            ).to(self.device)
        elif self.config.model_type == "ltx":
            self.model = self.config.model_class.from_single_file(
                self.config.model_path,
                torch_dtype=self.dtype
            ).to(self.device)
            self.model.enable_slicing()
            self.model.enable_tiling()

        print(f"Model loaded successfully from {self.config.model_path}.")

        self.transform = transforms.ToTensor()

    def preprocess_videos(self, video_paths: List[str]) -> Tuple[torch.Tensor, List[float], List[int], List[Tuple[int, int]]]:
        batch_frames = []
        fps_list, num_frames_list, resolutions = [], [], []

        for video_path in video_paths:
            video_reader = imageio.get_reader(video_path, "ffmpeg")
            meta_data = video_reader.get_meta_data()
            fps = meta_data.get('fps', 30)

            frames = [self.transform(frame) for frame in video_reader]
            video_reader.close()

            if not frames:
                raise ValueError(f"No frames found in video: {video_path}")

            num_frames = len(frames)
            resolution = frames[0].shape[1], frames[0].shape[2]

            fps_list.append(fps)
            num_frames_list.append(num_frames)
            resolutions.append(resolution)

            frames_tensor = torch.stack(frames).to(self.device).permute(1, 0, 2, 3)
            batch_frames.append(frames_tensor)

        batch_tensor = torch.stack(batch_frames).to(self.dtype)
        return batch_tensor, fps_list, num_frames_list, resolutions

    def encode(self, frames_tensor: torch.Tensor) -> torch.Tensor:
        if self.config.model_type in ["general","ltx"]:
            with torch.no_grad():
                encoded_frames = self.model.encode(frames_tensor)[0].sample()
        elif self.config.model_type == "cosmos":
            (encoded_frames,) = self.encoder.encode(frames_tensor)
        return encoded_frames

    def decode(self, encoded_tensor: torch.Tensor) -> torch.Tensor:
        if self.config.model_type in ["general","ltx"]:
            with torch.no_grad():
                decoded_frames = self.model.decode(encoded_tensor).sample
        elif self.config.model_type == "cosmos":
            decoded_frames = self.decoder.decode(encoded_tensor)
        return decoded_frames

    def save_videos(self, tensor: torch.Tensor, output_paths: List[str], fps_list: List[float],
                    num_frames_list: List[int], resolutions: List[Tuple[int, int]]):
        tensor = tensor.to(dtype=torch.float32)
        for i, (fps, num_frames, resolution, output_path) in enumerate(zip(fps_list, num_frames_list, resolutions, output_paths)):
            frames = tensor[i].permute(1, 2, 3, 0).cpu().numpy()
            frames = np.clip(frames, 0, 1) * 255
            frames = frames.astype(np.uint8)

            num_output_frames = frames.shape[0]
            assert num_output_frames == num_frames, (
                f"Frame count mismatch: input {num_frames} vs output {num_output_frames}")

            output_resolution = frames.shape[1], frames.shape[2]
            assert output_resolution == resolution, (
                f"Resolution mismatch: input {resolution} vs output {output_resolution}")

            writer = imageio.get_writer(output_path, fps=fps, codec='libx264')
            for frame in frames:
                writer.append_data(frame)
            writer.close()

            print(f"Saved video to {output_path} with {num_output_frames} frames at {output_resolution} resolution and {fps} fps.")

    def reconstruct_videos(self, video_paths: List[str], output_paths: List[str]):
        batch_size = self.config.batch_size
        for dataset_name, custom_size in self.config.custom_batch_size.items():
            if any(dataset_name in path for path in video_paths):
                batch_size = custom_size
                break

        for i in range(0, len(video_paths), batch_size):
            batch_video_paths = video_paths[i:i + batch_size]
            batch_output_paths = output_paths[i:i + batch_size]

            frames_tensor, fps_list, num_frames_list, resolutions = self.preprocess_videos(batch_video_paths)
            encoded = self.encode(frames_tensor)
            decoded = self.decode(encoded)
            self.save_videos(decoded, batch_output_paths, fps_list, num_frames_list, resolutions)
            del frames_tensor, encoded, decoded

In [9]:
# Testing code on V100 server with Cosmos
from diffusers import AutoencoderKLLTXVideo

# Processing Script
if __name__ == "__main__":
    config = Config(
        model_class=AutoencoderKLLTXVideo,  
        model_path="https://huggingface.co/Lightricks/LTX-Video/ltx-video-2b-v0.9.safetensors",   # they patch model into a single file
        device="cuda:1",
        dtype='float16',
        # "bfloat16",  # TODO Cosmos requires bfloat16
        batch_size=1,
        custom_batch_size={'imagenet_val': 1, 'textocr': 1, 'bridgedata_v2':1},  # Variable resolution
        source_base="/home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/processed_gt_v3",
        output_base="/home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/model_recon/ltx",
        model_type="ltx",  # Specify Cosmos as the model type
    )

    autoencoder = GeneralAutoEncoderKL(config)

    for dataset in os.listdir(config.source_base):
        dataset="ego-exo-4d-ego" # TODO: V100 can't work with large resolution above 520p
        dataset_source_path = os.path.join(config.source_base, dataset)
        dataset_output_path = os.path.join(config.output_base, dataset)

        if not os.path.isdir(dataset_source_path):
            continue

        print(f"Processing dataset: {dataset}")
        os.makedirs(dataset_output_path, exist_ok=True)

        video_files = sorted([f for f in os.listdir(dataset_source_path) if f.endswith('.mp4')])
        video_paths = [os.path.join(dataset_source_path, f) for f in video_files]
        output_paths = [os.path.join(dataset_output_path, f) for f in video_files]

        autoencoder.reconstruct_videos(video_paths, output_paths)

    print("All datasets have been processed successfully.")


Model loaded successfully from https://huggingface.co/Lightricks/LTX-Video/ltx-video-2b-v0.9.safetensors.
Processing dataset: ego-exo-4d-ego
Saved video to /home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/model_recon/ltx/ego-exo-4d-ego/0000.mp4 with 105 frames at (448, 448) resolution and 30.0 fps.
Saved video to /home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/model_recon/ltx/ego-exo-4d-ego/0002.mp4 with 105 frames at (448, 448) resolution and 30.0 fps.
Saved video to /home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/model_recon/ltx/ego-exo-4d-ego/0004.mp4 with 105 frames at (448, 448) resolution and 30.0 fps.
Saved video to /home/maij/fall_2024/sora3r/Open-Sora/data/vae_eval_bench/model_recon/ltx/ego-exo-4d-ego/0013.mp4 with 105 frames at (448, 448) resolution and 30.0 fps.


KeyboardInterrupt: 